In [1]:
import keras
import pickle
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
import os
import numpy as np
import glob
import random
import math
from keras.callbacks import ModelCheckpoint
import time
import keras.backend as K
import tensorflow as tf
from tensorflow import set_random_seed
import subprocess
from collections import Counter
import socket
import sys

Using TensorFlow backend.


In [2]:
argvv = ['./test']
seed_list = glob.glob('./neuzz_in/*')
seed_list.sort()
SPLIT_RATIO = len(seed_list)
rand_index = np.arange(SPLIT_RATIO)
np.random.shuffle(seed_list)

In [3]:
MAX_FILE_SIZE = 512
call=subprocess.check_output
raw_bitmap = {}
tmp_cnt = []
out = ''
for f in seed_list:
    tmp_list = []
    try:
        # append "-o tmp_file" to strip's arguments to avoid tampering tested binary.
        print(f)
        with open(f) as myinput:
            out = call(['./afl-showmap', '-q', '-e', '-o', '/dev/stdout', '-m', '512'] + argvv, stdin=myinput)
    except subprocess.CalledProcessError:
        print("find a crash")
    for line in out.splitlines():
        edge = line.split(':')[0]
        tmp_cnt.append(edge)
        tmp_list.append(edge)
    raw_bitmap[f] = tmp_list

./neuzz_in/id:000090,src:000088,op:int8,pos:41,val:+0,+cov
./neuzz_in/id:000027,src:000025,op:flip1,pos:10,+cov
./neuzz_in/id:000002,src:000000,op:flip4,pos:5
./neuzz_in/id:000029,src:000027,op:arith8,pos:11,val:+15,+cov
./neuzz_in/id:000083,src:000081,op:arith8,pos:38,val:+20,+cov
./neuzz_in/id:000070,src:000067,op:int8,pos:31,val:+0,+cov
./neuzz_in/id:000119,src:000117,op:flip2,pos:56,+cov
./neuzz_in/id:000110,src:000107,op:int8,pos:51,val:+0,+cov
./neuzz_in/id:000061,src:000059,op:flip4,pos:27,+cov
./neuzz_in/id:000109,src:000107,op:arith8,pos:51,val:+21,+cov
./neuzz_in/id:000036,src:000033,op:int8,pos:14,val:+0,+cov
./neuzz_in/id:000059,src:000057,op:flip2,pos:26,+cov
./neuzz_in/id:000011,src:000010,op:flip1,pos:2,+cov
./neuzz_in/id:000045,src:000043,op:arith8,pos:19,val:+21,+cov
./neuzz_in/id:000121,src:000119,op:arith8,pos:57,val:-14,+cov
./neuzz_in/id:000132,src:000130,op:int8,pos:62,val:+0,+cov
./neuzz_in/id:000133,src:000131,op:arith8,pos:63,val:+11,+cov
./neuzz_in/id:000031,s

In [4]:
counter = Counter(tmp_cnt).most_common()
label = [int(f[0]) for f in counter]
bitmap = np.zeros((len(seed_list), len(label)))
for idx,i in enumerate(seed_list):
    tmp = raw_bitmap[i]
    for j in tmp:
        if int(j) in label:
            bitmap[idx][label.index((int(j)))] = 1

In [5]:
fit_bitmap = np.unique(bitmap,axis=1)
MAX_BITMAP_SIZE = fit_bitmap.shape[1]
for idx,i in enumerate(seed_list):
    file_name = "./bitmaps/"+i.split('/')[-1]
    np.save(file_name,fit_bitmap[idx])

In [6]:
fit_label = []
for i in range(fit_bitmap.shape[1]):
    edges = []
    for j in range(bitmap.shape[1]):
        if (bitmap[:, j] == fit_bitmap[:, i]).all():
            edges.append(j)
    fit_label.append(edges)

In [7]:
def getIndex(edge):
    index = label.index(edge)
    for i, edges in enumerate(fit_label):
        if index in edges:
            return i
    return None

In [8]:
def accur_1(y_true,y_pred):
    y_true = tf.round(y_true)
    pred = tf.round(y_pred)
    summ = tf.constant(MAX_BITMAP_SIZE,dtype=tf.float32)
    wrong_num = tf.subtract(summ,tf.reduce_sum(tf.cast(tf.equal(y_true, pred),tf.float32),axis=-1))
    right_1_num = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(y_true,tf.bool), tf.cast(pred,tf.bool)),tf.float32),axis=-1)
    ret = K.mean(tf.divide(right_1_num,tf.add(right_1_num,wrong_num)))
    return ret

In [9]:
def build_model():
    batch_size = 32
    num_classes = MAX_BITMAP_SIZE
    epochs = 50
    model = Sequential()
    model.add(Dense(4096, input_dim=MAX_FILE_SIZE))
    model.add(Activation('relu'))
    model.add(Dense(num_classes))
    model.add(Activation('sigmoid'))
    opt = keras.optimizers.adam(lr=0.0001)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[accur_1])
    model.summary()
    return model

In [10]:
model = build_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4096)              2101248   
_________________________________________________________________
activation_1 (Activation)    (None, 4096)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 258)               1057026   
_________________________________________________________________
activation_2 (Activation)    (None, 258)               0         
Total params: 3,158,274
Trainable params: 3,158,274
Non-trainable params: 0
_________________________________________________________________


In [11]:
def generate_training_data(lb,ub):
    seed = np.zeros((ub-lb,MAX_FILE_SIZE))
    bitmap = np.zeros((ub-lb,MAX_BITMAP_SIZE))
    for i in range(lb,ub):
        tmp = open(seed_list[i],'r').read()
        ln = len(tmp)
        if ln < MAX_FILE_SIZE:
            tmp = tmp + (MAX_FILE_SIZE - ln) * '\0'
        seed[i-lb] = [ord(j) for j in list(tmp)]
    for i in range(lb,ub):
        file_name = "./bitmaps/"+ seed_list[i].split('/')[-1] + ".npy"
        bitmap[i-lb] = np.load(file_name)
    return seed,bitmap

In [12]:
def train_generate(batch_size):
    global seed_list
    while 1:
        np.random.shuffle(seed_list)
        for i in range(0,SPLIT_RATIO,batch_size):
            if (i+batch_size) > SPLIT_RATIO:
                x,y=generate_training_data(i,SPLIT_RATIO)
                x = x.astype('float32')/255
            else:
                x,y=generate_training_data(i,i+batch_size)
                x = x.astype('float32')/255
            yield (x,y)

In [13]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.lr = []
    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.lr.append(step_decay(len(self.losses)))
        print(step_decay(len(self.losses)))

In [14]:
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.7
    epochs_drop = 10.0
    lrate = initial_lrate * math.pow(drop,math.floor((1+epoch)/epochs_drop))
    return lrate

In [19]:
def train(model):
    loss_history = LossHistory()
    lrate = keras.callbacks.LearningRateScheduler(step_decay)
    callbacks_list = [loss_history, lrate]
    model.fit_generator(train_generate(8),
              steps_per_epoch = (SPLIT_RATIO/8 + 1),
              epochs=100,
              verbose=1, callbacks=callbacks_list)
    # Save model and weights
    model.save_weights("hard_label.h5")

In [20]:
train(model)

Epoch 1/100
17/17 [==============================] - 0s 26ms/step - loss: 0.0767 - accur_1: 0.8526
0.001
Epoch 2/100
17/17 [==============================] - 0s 28ms/step - loss: 0.0790 - accur_1: 0.8502
0.001
Epoch 3/100
17/17 [==============================] - 0s 27ms/step - loss: 0.0741 - accur_1: 0.8585
0.001
Epoch 4/100
17/17 [==============================] - 0s 26ms/step - loss: 0.0751 - accur_1: 0.8595
0.001
Epoch 5/100
17/17 [==============================] - 0s 26ms/step - loss: 0.0736 - accur_1: 0.8500
0.001
Epoch 6/100
17/17 [==============================] - 0s 27ms/step - loss: 0.0803 - accur_1: 0.8414
0.001
Epoch 7/100
17/17 [==============================] - 0s 29ms/step - loss: 0.0897 - accur_1: 0.8342
0.001
Epoch 8/100
17/17 [==============================] - 0s 28ms/step - loss: 0.0716 - accur_1: 0.8544
0.001
Epoch 9/100
17/17 [==============================] - 0s 28ms/step - loss: 0.0685 - accur_1: 0.8624
0.0007
Epoch 10/100
17/17 [==============================] - 

17/17 [==============================] - 0s 29ms/step - loss: 0.0366 - accur_1: 0.9298
8.23543e-05
Epoch 77/100
17/17 [==============================] - 0s 29ms/step - loss: 0.0368 - accur_1: 0.9290
8.23543e-05
Epoch 78/100
17/17 [==============================] - 1s 30ms/step - loss: 0.0369 - accur_1: 0.9274
8.23543e-05
Epoch 79/100
17/17 [==============================] - 0s 28ms/step - loss: 0.0363 - accur_1: 0.9298
5.764801e-05
Epoch 80/100
17/17 [==============================] - 0s 27ms/step - loss: 0.0362 - accur_1: 0.9290
5.764801e-05
Epoch 81/100
17/17 [==============================] - 0s 28ms/step - loss: 0.0361 - accur_1: 0.9307
5.764801e-05
Epoch 82/100
17/17 [==============================] - 0s 29ms/step - loss: 0.0361 - accur_1: 0.9304
5.764801e-05
Epoch 83/100
17/17 [==============================] - 1s 30ms/step - loss: 0.0361 - accur_1: 0.9302
5.764801e-05
Epoch 84/100
17/17 [==============================] - 1s 30ms/step - loss: 0.0363 - accur_1: 0.9282
5.764801e-05

In [23]:
def vectorize_file(fl, isfile, vectorize=True):
    seed = np.zeros((1,MAX_FILE_SIZE))
    if isfile:
        tmp = open(fl,'r').read()
    else:
        tmp = fl
    ln = len(tmp)
    if ln < MAX_FILE_SIZE:
        tmp = tmp + (MAX_FILE_SIZE - ln) * '\0'
    seed[0] = [ord(j) for j in list(tmp)]
    if vectorize:
        seed = seed.astype('float32')/255
    return seed

In [25]:
def gradient(model, edge, seed, isfile, vectorize=True):
    layer_list = [(layer.name, layer) for layer in model.layers]
    index = getIndex(edge)
    loss = layer_list[-2][1].output[:,index]
    grads = K.gradients(loss,model.input)[0]
    iterate = K.function([model.input], [loss, grads])
    x=vectorize_file(seed, isfile, vectorize)
    loss_value, grads_value = iterate([x])
    idx = np.flip(np.argsort(np.absolute(grads_value),axis=1)[:, -MAX_FILE_SIZE:].reshape((MAX_FILE_SIZE,)),0)
    val = np.sign(grads_value[0][idx])
    return idx, val

In [29]:
idx, val = gradient(model, 137, "xhqxxxxx", False, True)
idx[:32]

array([ 2,  9,  0,  8,  7, 58,  4, 19,  6, 33, 34, 11, 60, 12,  5, 30, 15,
       56, 20, 16, 35,  1, 26, 31, 28, 61, 21, 18, 10, 32, 59, 24])

In [30]:
idx, val = gradient(model, 137, "xhexxxxx", False, False)
idx[:32]

array([ 2,  0,  9,  8,  7, 58,  4, 19,  6, 33, 34, 11, 60,  5, 15, 30, 12,
       20, 56, 16,  1, 35, 26, 18, 31, 21, 61, 10, 59, 28, 32, 24])

In [42]:
idx, val = gradient(model, 46697, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", True, True)
idx[:64]

array([ 51,  45,  55,  44,  42,  46,  49,  38,  58,  43,  28,  40,  59,
        56,  62,  63,  52,  61,  10,  39,  50,   0,   9,   8,  11,  41,
        54,  16,  18,   2,   1,  53,  14,  17,  23,  19,  20,  21,  25,
        22,  15,  24,  13,   3,  27,  12,  30,  36,  34,  32,   7,  57,
        29,  48,  82,   5,   6,   4,  35, 105,  31,  26, 315, 480])

In [33]:
idx, val = gradient(model, 46697, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", True, False)
idx[:32]

array([58, 51, 56, 55, 45, 60,  9,  0, 19, 33,  2, 53, 44, 59, 46, 43,  7,
       34, 42, 32, 20, 35, 50, 26,  1,  5, 49, 24, 21, 57, 41, 54])

In [36]:
def evaluate(model, seed, vectorize=False):
    x = vectorize_file(seed, True, vectorize)
    file_name = file_name = "./bitmaps/"+ seed.split('/')[-1] + ".npy"
    return model.predict(x), np.load(file_name)

In [39]:
y, ty = evaluate(model, "neuzz_in/id:000134,src:000131,op:int8,pos:63,val:+0,+cov", False)

In [40]:
y

array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
        0.000000e+00, 0.000000e+00, 0.00

In [41]:
ty

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.